In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("world_happiness_score.csv")
df.head(2)

Country  Happiness_Rank  Happiness_Score  year
0  Switzerland               1            7.587  2015
1      Iceland               2            7.561  2015

### [[ 1번 ]]

In [3]:
df_2015 = df.loc[df["year"] == 2015, ["Country", "year"]]
df_2016 = df.loc[df["year"] == 2016, ["Country", "year"]]
df_2017 = df.loc[df["year"] == 2017, ["Country", "year"]]

In [4]:
df_join = df_2015.merge(df_2016, left_on = "Country", right_on = "Country", how = "inner")
df_join = df_join.merge(df_2017, left_on = "Country", right_on = "Country", how = "inner")
df_join.head(2)

Country  year_x  year_y  year
0  Switzerland    2015    2016  2017
1      Iceland    2015    2016  2017

In [5]:
df["Country"].nunique() - len(df_join)

20

### [[ 2번 ]]

In [6]:
df_2 = df.loc[df["Country"].isin(df_join["Country"]), ]
print(len(df_2))
print(len(df))

438
470


In [7]:
df_2.head(2)

Country  Happiness_Rank  Happiness_Score  year
0  Switzerland               1            7.587  2015
1      Iceland               2            7.561  2015

In [8]:
df_2_wide = df_2.pivot(index = "Country", columns = "year", values = "Happiness_Score")
df_2_wide = df_2_wide.reset_index()
df_2_wide.head(2)

year      Country   2015   2016   2017
0     Afghanistan  3.575  3.360  3.794
1         Albania  4.959  4.655  4.644

In [9]:
df_2_wide["HS_ir"] = (df_2_wide[2017] - df_2_wide[2015]) / 2
df_2_wide = df_2_wide.sort_values("HS_ir", ascending = False)
df_2_wide.head(3)

year  Country   2015   2016   2017   HS_ir
71     Latvia  5.098  5.560  5.850  0.3760
108   Romania  5.124  5.528  5.825  0.3505
129      Togo  2.839  3.303  3.495  0.3280

### [[ 3번 ]]

In [10]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [11]:
model = ols(formula = "Happiness_Score ~ C(year)", data = df_2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Happiness_Score   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                  0.004277
Date:                Mon, 05 Jul 2021   Prob (F-statistic):              0.996
Time:                        11:06:10   Log-Likelihood:                -678.98
No. Observations:                 438   AIC:                             1364.
Df Residuals:                     435   BIC:                             1376.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.3959      0.095     56.983      0.000       5.210       5.582
C(year)[T.2016]    -0.0112      0.134     -0.084      0.933      -0.274       0.252
C(year)[T.2017]    -0.0010      0.134     -0.007      0.994      -0.264       0.262
==============================================================================
Omnibus:                       41.991   Durbin-Watson:                   0.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               13.285
Skew:                           0.050   Prob(JB):                      0.00130
Kurtosis:                       2.153   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""